# Checkpoints

This document examines how to save and restore TensorFlow models built with Estimators. TensorFlow provides two model formats:

- checkpoints, which is a format dependent on the code that created the model.
- SavedModel, which is a format independent of the code that created the model.

This document focuses on checkpoints. For details on SavedModel, see the [Saving and Restoring](https://tensorflow.google.cn/programmers_guide/saved_model) chapter of the *TensorFlow Programmer's Guide*.

## Saving partially-trained models
Estimators automatically write the following to disk:

- checkpoints, which are versions of the model created during training.
- event files, which contain information that [TensorBoard](https://developers.google.cn/machine-learning/glossary/#TensorBoard) uses to create visualizations.

To specify the top-level directory in which the Estimator stores its information, assign a value to the optional model_dir argument of any Estimator's constructor. For example, the following code sets the model_dir argument to the models/iris directory:

In [3]:
import iris_data
import tensorflow as tf

(train_x, train_y), (test_x, test_y) = iris_data.load_data()
    
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3,
    model_dir='models/iris')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AB820D8400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Suppose you call the Estimator's train method. For example:

In [5]:
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, batch_size=100),
    steps=200)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/iris\model.ckpt.
INFO:tensorflow:loss = 235.59146, step = 1
INFO:tensorflow:global_step/sec: 490.445
INFO:tensorflow:loss = 70.60144, step = 101 (0.203 sec)
INFO:tensorflow:Saving checkpoints for 200 into models/iris\model.ckpt.
INFO:tensorflow:Loss for final step: 51.913696.


As suggested by the following diagrams, the first call to train adds checkpoints and other files to the model_dir directory:

![](https://tensorflow.google.cn/images/first_train_calls.png)
<center>The first call to train().</center>

To see the objects in the created model_dir directory on a UNIX-based system, just call ls as follows:

In [7]:
!bash -c "ls -1 models/iris"

checkpoint
events.out.tfevents.1529404546.USER-20150123VJ
graph.pbtxt
model.ckpt-1.data-00000-of-00001
model.ckpt-1.index
model.ckpt-1.meta
model.ckpt-200.data-00000-of-00001
model.ckpt-200.index
model.ckpt-200.meta


The preceding ls command shows that the Estimator created checkpoints at steps 1 (the start of training) and 200 (the end of training).

## Default checkpoint directory

If you don't specify model_dir in an Estimator's constructor, the Estimator writes checkpoint files to a temporary directory chosen by Python's [tempfile.mkdtemp](https://docs.python.org/3/library/tempfile.html#tempfile.mkdtemp) function. For example, the following Estimator constructor does not specify the model_dir argument:

In [10]:
classifer = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3)

print(classifier.model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmpv3azjnld', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AB836F5198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
models/iris


The tempfile.mkdtemp function picks a secure, temporary directory appropriate for your operating system. For example, a typical temporary directory on macOS might be something like the following:

## Checkpointing Frequency

By default, the Estimator saves [checkpoints](https://developers.google.cn/machine-learning/glossary/#checkpoint) in the `model_dir` according to the following schedule:

- Writes a checkpoint every 10 minutes (600 seconds).
- Writes a checkpoint when the `train` method starts (first iteration) and completes (final iteration).
- Retains only the 5 most recent checkpoints in the directory.

You may alter the default schedule by taking the following steps:

- Create a [RunConfig](https://tensorflow.google.cn/api_docs/python/tf/estimator/RunConfig) object that defines the desired schedule.
- When instantiating the Estimator, pass that `RunConfig` object to the Estimator's `config` argument.

For example, the following code changes the checkpointing schedule to every 20 minutes and retains the 10 most recent checkpoints:

In [12]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 20*60,  # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 10,        # Retain the 10 most recent checkpoints.
)

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3,
    model_dir='models/iris',
    config=my_checkpointing_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1200, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AB836F56A0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Restoring your model

The first time you call an Estimator's train method, TensorFlow saves a checkpoint to the model_dir. Each subsequent call to the Estimator's train, evaluate, or predict method causes the following:

1. The Estimator builds the model's [graph](https://developers.google.cn/machine-learning/glossary/#graph) by running the model_fn(). (For details on the model_fn(), see [Creating Custom Estimators.](https://tensorflow.google.cn/get_started/custom_estimators))
2. The Estimator initializes the weights of the new model from the data stored in the most recent checkpoint.

In other words, as the following illustration suggests, once checkpoints exist, TensorFlow rebuilds the model each time you call `train()`, `evaluate()`, or `predict()`.

![](https://tensorflow.google.cn/images/subsequent_calls.png)
<center>Subsequent calls to train(), evaluate(), or predict()</center>

## Avoiding a bad restoration

Restoring a model's state from a checkpoint only works if the model and checkpoint are compatible. For example, suppose you trained a DNNClassifier Estimator containing two hidden layers, each having 10 nodes:

In [15]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3,
    model_dir='models/iris')

classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, batch_size=100),
        steps=200)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AB836ACFD0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints 

After training (and, therefore, after creating checkpoints in models/iris), imagine that you changed the number of neurons in each hidden layer from 10 to 20 and then attempted to retrain the model:

In [19]:
classifier2 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[20, 20],
    n_classes=3,
    model_dir='models/iris')

classifier2.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, batch_size=100),
        steps=200)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AB82383128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris\model.ckpt-600


InvalidArgumentError: tensor_name = dnn/hiddenlayer_0/bias; shape in shape_and_slice spec [20] does not match the shape stored in checkpoint: [10]
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "C:\Users\Administrator\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Administrator\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Administrator\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()

Since the state in the checkpoint is incompatible with the model described in classifier2, retraining fails with the following error:

To run experiments in which you train and compare slightly different versions of a model, save a copy of the code that created each model_dir, possibly by creating a separate git branch for each version. This separation will keep your checkpoints recoverable.

## Summary

Checkpoints provide an easy automatic mechanism for saving and restoring models created by Estimators.

See the [Saving and Restoring](https://tensorflow.google.cn/programmers_guide/saved_model) chapter of the *TensorFlow Programmer's Guide* for details on:

- Saving and restoring models using low-level TensorFlow APIs.
- Exporting and importing models in the SavedModel format, which is a language-neutral, recoverable, serialization format.